In [1]:
import torch
import torchvision
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

/home/ubuntu/anaconda3/envs/cs231n/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torchvision.__version__

'0.12.0'

In [3]:
data = torchvision.datasets.Sintel(".")

In [4]:
class TSintel(torchvision.datasets.Sintel):
    def __init__(self, root):
        super().__init__(root=root)
        
    def __getitem__(self, index):
        img1, img2, flow = super().__getitem__(index)
        img1 = torchvision.transforms.ToTensor()(img1)
        img2 = torchvision.transforms.ToTensor()(img2)
        return img1, img2, flow

In [5]:
Tdata = TSintel(".")

In [6]:
train_size = round(len(Tdata) * 0.8)
test_size = round(len(Tdata) * 0.2)

In [7]:
assert train_size + test_size == len(Tdata)

In [8]:
train_data, test_data = torch.utils.data.random_split(Tdata, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [9]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

In [10]:
epochs = 10
lr = 1e-3

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [12]:
from model import FlowNetS
from multiscaleloss import multiscaleEPE

/home/ubuntu/cs231n/util.py:11: ImportWarning: failed to load custom correlation modulewhich is needed for FlowNetC
  warnings.warn("failed to load custom correlation module"


In [13]:
model = FlowNetS()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model = model.to(device)

In [32]:
train_params = {
    "epochs": 10,
    "lr": 1e-3,
}

dataloaders = {
    "train": train_loader,
    "val": test_loader,
}

In [36]:
len(train_loader.dataset)

833

In [33]:
def train_flownet(dataloaders, train_params):
    train_loader = dataloaders.get("train")
    val_loader = dataloaders.get("val")
    
    epochs = train_params.get("epochs")
    lr = train_params.get("lr")
    
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = FlowNetS()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model = model.to(device)
    
    losses = {
        "train": [],
        "val": []
    }
    
    model.train()
    for epoch in range(epochs):
    
        print("Epoch", str(epoch) + ": ", end="")
        train_loss = 0.0
        val_loss = 0.0

        for i, (img1, img2, label) in tqdm(enumerate(train_loader)):
            image = torch.cat((img1, img2), dim=1).to(device)
            label = label.to(device)

            optimizer.zero_grad()
            outputs = model(image)
            h, w = label.size()[-2:]
            outputs = [torch.nn.functional.interpolate(outputs[0], (h,w)), *outputs[1:]]

            loss = multiscaleEPE(outputs, label, sparse=False)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_loss += realEPE(outputs[0], label, sparse=False).item()
            
        with torch.no_grad():
            model.eval()
            val_loss = 0.0

            for i, (img1, img2, target) in enumerate(val_loader):
                target = target.to(device)
                img = torch.cat((img1, img2), dim=1).to(device)

                # compute output
                output = model(img)
                loss = realEPE(output, target, sparse=False)
                val_loss += loss.item()
                
        val_loss /= len(val_loader.dataset)

        print("Train Loss", train_loss, "Val Loss", val_loss)
            losses["train"].append(train_loss)
            losses["val"].append(val_loss)
            
    return losses

In [34]:
flownet_losses = train_flownet(dataloaders, train_params)

Epoch 0: 

53it [00:59,  1.12s/it]


Train Loss tensor(729.1456, device='cuda:0') Val Loss 204.21940517425537


KeyError: 'test'

In [19]:
def valid_flownet(val_loader):
    with torch.no_grad():
        model.eval()
        val_loss = 0.0

        for i, (img1, img2, target) in enumerate(val_loader):
            target = target.to(device)
            img = torch.cat((img1, img2), dim=1).to(device)

            # compute output
            output = model(img)
            loss = realEPE(output, target, sparse=False)
            val_loss += loss.item()

        

In [20]:
from multiscaleloss import *
valid_flownet(test_loader)

tensor(9.6814, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(18.1288, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(8.9003, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(16.1328, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(11.8275, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(19.8937, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(11.0718, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(12.1404, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9.6635, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(19.0069, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(16.6993, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(20.7556, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(18.4889, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
losses = []

for epoch in range(epochs):
    
    print("Epoch", str(epoch) + ": ", end="")
    epoch_loss = 0.0
    
    for i, (img1, img2, label) in tqdm(enumerate(train_loader)):
        image = torch.cat((img1, img2), dim=1)
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        outputs = model(image)
        h, w = label.size()[-2:]
        outputs = [torch.nn.functional.interpolate(outputs[0], (h,w)), *outputs[1:]]
        
        loss = multiscaleEPE(outputs, label, sparse=True)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print("Loss:", epoch_loss)
    losses.append(epoch_loss)

In [ ]:
i = 160
output = model(torch.cat((Tdata[i][0], Tdata[i][1]), dim=0).unsqueeze(dim=0).to(device))
img_size = Tdata[i][0].shape[1:]
output = torch.nn.functional.interpolate(output, size=img_size, mode="bilinear", align_corners=False).squeeze()

In [ ]:
data[i][0]

In [ ]:
torchvision.transforms.ToPILImage()(torchvision.utils.flow_to_image(output))